In [1]:
# WORKING CODE FOR DAILY .NC WITH 49 TIMESTAMPS,ACCUMULATED DAILY CSV FILES
# (CODE FOR LOG FILE:Log_2022.txt)
import os
import glob
import xarray as xr
import cftime
import pandas as pd
import numpy as np

# Define the base paths
imerg_base_path = "/home/pc/Desktop/IMERG_2022"
output_base_path = "/home/pc/Desktop/IMERG_OP_2022"
log_file_path = os.path.join(output_base_path, "log_2022.txt")

# Create output directory if it doesn't exist
os.makedirs(output_base_path, exist_ok=True)

# Define the months you want to process
months_to_process = range(3, 11)

# Define the years you want to process
years_to_process = range(2022, 2023)

# Helper function to determine the number of days in a month
def days_in_month(year, month):
    if month in [1, 3, 5, 7, 8, 10, 12]:
        return 31
    elif month in [4, 6, 9, 11]:
        return 30
    elif month == 2:
        # Check for leap year
        if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
            return 29
        else:
            return 28
    else:
        return 0

# Define rain event categories
rain_ranges = {
    'Background rain': (0, 7.8),
    'Moderate Rain Events': (7.8, 39.4),
    'Intense Rain Events': (39.4, 136.6),
    'Extreme Rain Events': (136.6, np.inf)
}

# Function to determine the type of rain event
def determine_rain_type(precipitation):
    for event, (min_val, max_val) in rain_ranges.items():
        if min_val <= precipitation < max_val:
            return event
    return 'Unknown'

# Open the log file
with open(log_file_path, 'w') as log_file:
    # Write the title to the log file
    log_file.write("ACCUMULATED RAIN CSV AND .NC FOR YEAR 2022\n\n")
    
    # Loop over the specified years
    for iyear in years_to_process:
        # Loop over the specified months
        for imonth in months_to_process:
            # Determine the number of days in the month
            dayend = days_in_month(iyear, imonth)
            
            # Loop over the days in the month
            for iday in range(1, dayend + 1):
                # Skip days before March 31
                if iyear == 2022 and imonth == 3 and iday < 31:
                    continue
                
                # Create a list to hold the datasets
                datasets = []
                
                # Gather files for the current day and the next day
                for iday_offset in [0, 1]:
                    current_day = iday + iday_offset
                    
                    if current_day > dayend:
                        # Move to the next month if the day exceeds the current month
                        next_month = imonth + 1
                        next_day = current_day - dayend
                        if next_month > 10:
                            next_year = iyear + 1
                            next_month = 1
                        else:
                            next_year = iyear
                    else:
                        next_year = iyear
                        next_month = imonth
                        next_day = current_day
                        
                    log_file.write(f"{iyear}-{imonth:02d}-{iday:02d}:\n")
                    log_file.write(f"{iday_offset}-{current_day:02d}:\n")
                    log_file.write(f"{next_year}-{next_month:02d}-{next_day:02d}:\n")
                    # Construct the file prefix
                    file_prefix = f'3B-HHR.MS.MRG.3IMERG.{next_year}{next_month:02d}{next_day:02d}'

                    # Find all files with the matching prefix
                    file_pattern = os.path.join(imerg_base_path, f"{file_prefix}*.HDF5.nc4")
                    matching_files = glob.glob(file_pattern)
                    
                    if not matching_files:
                        log_file.write(f"No files found for prefix: {file_prefix}\n")
                        continue
                    
                    # Load each file (example loading using xarray)
                    for file_name in sorted(matching_files):
                        try:
                            ds = xr.open_dataset(file_name)
                            datasets.append(ds)
                        except Exception as e:
                            log_file.write(f"Error loading {file_name}: {str(e)}\n")
                    
                # Concatenate datasets along the time dimension
                if datasets:
                    combined_ds = xr.concat(datasets, dim='time')
                    
                    # Define time slicing parameters
                    start_time = cftime.DatetimeJulian(2022, imonth, iday, 3, 0, 0)
                    end_time = cftime.DatetimeJulian(next_year, next_month, next_day, 3, 0, 0)
                    
                    # Perform time slicing
                    ds_sliced = combined_ds.sel(time=slice(start_time, end_time))
                    
                    # Verify the number of time steps
                    time_steps = len(ds_sliced.time)
                    if time_steps != 49:
                        log_file.write(f"Warning: Expected 49 time steps but found {time_steps} for {start_time} to {end_time}. There might be missing data.\n")

                    # Log the time range
                    log_file.write(f"Processing time range: {start_time} to {end_time}\n")
                    
                    # Save the accumulated dataset as a .nc file for verification
                    accumulated_nc_file = os.path.join(output_base_path, f"Acc_rainfall_3gmt_{start_time.strftime('%Y%m%d')}_3gmt_{end_time.strftime('%Y%m%d')}.nc")
                    try:
                        ds_sliced.to_netcdf(accumulated_nc_file)
                        log_file.write(f"Accumulated dataset saved to: {accumulated_nc_file}\n")
                    except Exception as e:
                        log_file.write(f"Error saving {accumulated_nc_file}: {str(e)}\n")

                    # Open the .nc file and log the contents
                    try:
                        with xr.open_dataset(accumulated_nc_file) as nc_file:
                            log_file.write(f"Contents of {accumulated_nc_file}:\n")
                            log_file.write(str(nc_file))
                            log_file.write('\n\n')
                    except Exception as e:
                        log_file.write(f"Error opening {accumulated_nc_file}: {str(e)}\n")

                    # Calculate and save daily accumulated precipitation
                    try:
                        accumulated_precipitation = ds_sliced['precipitation'].sum(dim='time', skipna=True)
                        accumulated_df = accumulated_precipitation.to_dataframe().reset_index()

                        # Add rain type column
                        accumulated_df['Rain Type'] = accumulated_df['precipitation'].apply(determine_rain_type)
                        accumulated_df['Date'] = end_time.strftime('%Y-%m-%d')

                        # Define output CSV file path
                        output_date = end_time.strftime('%Y-%m-%d')
                        accumulated_csv_file = os.path.join(output_base_path, f"Acc_rainfall_3gmt_{start_time.strftime('%Y-%m-%d')}_3gmt_{end_time.strftime('%Y-%m-%d')}.csv")
                        
                        # Save the CSV
                        accumulated_df.to_csv(accumulated_csv_file, index=False)
                        log_file.write(f"Saved accumulated precipitation to CSV: {accumulated_csv_file}\n")
                    except Exception as e:
                        log_file.write(f"Error calculating or saving CSV for {start_time} to {end_time}: {str(e)}\n")


                    # Close the datasets
                    combined_ds.close()
                    for ds in datasets:
                        ds.close()

    log_file.write("Processing completed.\n")

In [3]:
# DAILY_ACC_RAIN_.NC FILES ( THE SAME .NC FILES IS CREATED FROM THE ABOVE CODE)
#(CODE FOR LOG FILE:Log_2022_Daily_Acc_Rain.txt) 
import os
import glob
import xarray as xr
import cftime

# Define the base paths
imerg_base_path = "/home/pc/Desktop/IMERG_2022"
output_base_path = "/home/pc/Desktop/IMERG_OP_2022"
log_file_path = os.path.join(output_base_path, "log_2022_Daily_Acc_Rain.txt")

# Create output directory if it doesn't exist
os.makedirs(output_base_path, exist_ok=True)

# Define the months you want to process
months_to_process = range(3, 11)

# Define the years you want to process
years_to_process = range(2022, 2023)

# Helper function to determine the number of days in a month
def days_in_month(year, month):
    if month in [1, 3, 5, 7, 8, 10, 12]:
        return 31
    elif month in [4, 6, 9, 11]:
        return 30
    elif month == 2:
        # Check for leap year
        if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
            return 29
        else:
            return 28
    else:
        return 0

# Function to perform time slicing and accumulation
def slice_and_accumulate(iyear, imonth, iday, log_file):
    datasets = []
    for iday_offset in [0, 1]:
        current_day = iday + iday_offset
        if current_day > days_in_month(iyear, imonth):
            next_month = imonth + 1
            next_day = current_day - days_in_month(iyear, imonth)
            if next_month > 10:
                next_year = iyear + 1
                next_month = 1
            else:
                next_year = iyear
        else:
            next_year = iyear
            next_month = imonth
            next_day = current_day

        log_file.write(f"{iyear}-{imonth:02d}-{iday:02d}:\n")
        log_file.write(f"{iday_offset}-{current_day:02d}:\n")

        file_prefix = f'3B-HHR.MS.MRG.3IMERG.{next_year}{next_month:02d}{next_day:02d}'
        file_pattern = os.path.join(imerg_base_path, f"{file_prefix}*.HDF5.nc4")
        matching_files = glob.glob(file_pattern)

        for file_name in sorted(matching_files):
            try:
                ds = xr.open_dataset(file_name)
                datasets.append(ds)
            except Exception as e:
                log_file.write(f"Error loading {file_name}: {str(e)}\n")

    if datasets:
        combined_ds = xr.concat(datasets, dim='time')
        start_time = cftime.DatetimeJulian(iyear, imonth, iday, 3, 0, 0)
        end_time = cftime.DatetimeJulian(next_year, next_month, next_day, 3, 0, 0)
        ds_sliced = combined_ds.sel(time=slice(start_time, end_time))

        time_steps = len(ds_sliced.time)
        if time_steps != 49:
            log_file.write(f"Warning: Expected 49 time steps but found {time_steps} for {start_time} to {end_time}. There might be missing data.\n")

        try:
            accumulated_precipitation = ds_sliced['precipitation'].sum(dim='time', skipna=True)
            accumulated_precipitation = accumulated_precipitation.expand_dims('time')
            accumulated_precipitation_ds = accumulated_precipitation.to_dataset(name='accumulated_precipitation')
            accumulated_precipitation_ds['time'] = [end_time.strftime('%Y-%m-%d')]

            daily_nc_file = os.path.join(output_base_path, f"IMERG_Daily_acc_rainfall_{end_time.strftime('%Y-%m-%d')}.nc")
            accumulated_precipitation_ds.to_netcdf(daily_nc_file)
            log_file.write(f"Saved daily accumulated precipitation to .nc: {daily_nc_file}\n")

            # Open the .nc file and log the contents and accumulated precipitation values
            try:
                with xr.open_dataset(daily_nc_file) as nc_file:
                    log_file.write(f"Contents of {daily_nc_file}:\n")
                    log_file.write(str(nc_file))
                    log_file.write('\n')
                    log_file.write("Accumulated Precipitation Values:\n")
                    log_file.write(str(nc_file['accumulated_precipitation'].values))
                    log_file.write('\n\n')
            except Exception as e:
                log_file.write(f"Error opening {daily_nc_file}: {str(e)}\n")

        except Exception as e:
            log_file.write(f"Error calculating or saving .nc file for {end_time.strftime('%Y-%m-%d')}: {str(e)}\n")

        # Log the time range with timestamps
        log_file.write("Timestamps:\n")
        log_file.write(f"Processing time range: {start_time} to {end_time} ({start_time.strftime('%Y-%m-%d %H:%M:%S')} to {end_time.strftime('%Y-%m-%d %H:%M:%S')})\n")

        for timestamp in ds_sliced.time.values:
            log_file.write(f"{timestamp}\n")

        combined_ds.close()
        for ds in datasets:
            ds.close()

with open(log_file_path, 'w') as log_file:
    log_file.write("DAILY_ACCUMULATED_RAINFALL.NC FOR YEAR 2022\n\n")

    for iyear in years_to_process:
        for imonth in months_to_process:
            dayend = days_in_month(iyear, imonth)

            for iday in range(1, dayend + 1):
                if iyear == 2022 and imonth == 3 and iday < 31:
                    continue

                slice_and_accumulate(iyear, imonth, iday, log_file)

    log_file.write("Processing completed.\n")

In [4]:
# CONCATENATED DAILY ACCUMULATED .NC FILES INTO MONTHLY ACCUMULATED RAINFALLDATA IN .NC 
#(CODE FOR LOG FILE:Log_2022_Monthly_accumulated.txt) 
import xarray as xr
import pandas as pd
import os

# Define the directory containing the daily .nc files
directory = "/home/pc/Desktop/IMERG_OP_2022"
output_base_path = directory
log_file_path = os.path.join(output_base_path, "log_2022_Monthly_accumulated.txt")

# Open the log file
with open(log_file_path, 'w') as log_file:
    # Write the title to the log file
    log_file.write("MONTHLY_ACCUMULATED_YEAR 2022\n\n")

    # Define the range of months to process
    months = ["04", "05", "06", "07", "08", "09", "10"]
    year = "2022"

    # Function to process each month
    def process_month(month):
        daily_datasets = []

        # Determine the number of days in the month
        num_days = pd.Period(year + "-" + month).days_in_month
        log_file.write(f"Processing month: {month}, Number of days: {num_days}\n")

        # Load daily files for the given month
        for day in range(1, num_days + 1):
            file_path = os.path.join(directory, f"IMERG_Daily_acc_rainfall_{year}-{month}-{day:02d}.nc")
            log_file.write(f"Opening file: {file_path}\n")
            
            try:
                with xr.open_dataset(file_path) as nc_file:
                    log_file.write(f"Contents of {file_path}:\n{nc_file}\n")
                    daily_datasets.append(nc_file)
            except Exception as e:
                log_file.write(f"Failed to open file {file_path}: {e}\n")
                continue

        # Combine daily datasets into a single monthly dataset
        log_file.write(f"Combining daily datasets for month: {month}\n")
        monthly_ds = xr.concat(daily_datasets, dim="time")

        # Ensure the time dimension remains consistent
        time_values = [pd.to_datetime(str(t.values)) for t in monthly_ds['time']]
        monthly_ds['time'] = time_values
        log_file.write(f"Updated time dimension for month: {month}\n")

        # Rename the time dimension to day
        monthly_ds = monthly_ds.rename({'time': 'day'})
        log_file.write(f"Renamed time dimension to day for month: {month}\n")

        # Save the combined dataset to a new .nc file
        output_path = os.path.join(directory, f"Daily_3gmt_to_3gmt_accumulated_rainfall_{year}-{month}_{1}-{num_days}.nc")
        monthly_ds.to_netcdf(output_path)
        log_file.write(f"Saved combined dataset to: {output_path}\n")

        # Log the saved file and its contents
        log_file.write(f"Opening and logging contents of {output_path}\n")
        try:
            with xr.open_dataset(output_path) as nc_file:
                log_file.write(f"Contents of {output_path}:\n{nc_file}\n")
                log_file.write("Accumulated Precipitation Values:\n")
                
                # Assuming 'accumulated_precipitation' is the variable name
                if 'accumulated_precipitation' in nc_file:
                    log_file.write(str(nc_file['accumulated_precipitation'].values))
                else:
                    log_file.write("Variable 'accumulated_precipitation' not found in the dataset.\n")
                
                log_file.write('\n')
        except Exception as e:
            log_file.write(f"Error opening and logging contents of {output_path}: {e}\n")

    # Process each month in the specified range
    for month in months:
        process_month(month)

    log_file.write("All months processed.\n")
